# Model and Evaluation


## Objectives


- Implement ML pipeline. 
- Implement regression model to predict the sale price of properties,


## Inputs 


- outputs/datasets/datacollection/HousePrices.csv


## Outputs


- Train Set and Test Set. 
- Machine Learning pipeline. 


## CRISP-DM


- "Modelling and Evaluation".

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues'

In [4]:
import numpy as np
import pandas as pd 
df = pd.read_csv("outputs/datasets/datacollection/HousePrices.csv")

print(df.shape)
df.head(5)

(1460, 24)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


In [5]:
from sklearn.pipeline import Pipeline

### Data Cleaning
from feature_engine.imputation import MeanMedianImputer
from feature_engine.selection import DropFeatures
from feature_engine.imputation import CategoricalImputer

### Feature Engineering
from feature_engine import creation
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer

### Feature Scaling
from sklearn.preprocessing import StandardScaler

### Feature Selection
from sklearn.feature_selection import SelectFromModel

### ML algorithms 
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [7]:
def PipelineOptimization(model):
    pipeline_base = Pipeline([

    # Data Cleaning

    ("ArbitraryNumberImputer",ArbitraryNumberImputer(arbitrary_number=0,
                                variables = ['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF']) ),

    ("CategoricalEncoder",CategoricalImputer(imputation_method='missing',fill_value='Unf',
                                variables = ['BsmtFinType1', 'GarageFinish'])),

    ("MeanMedianImputer",MeanMedianImputer(imputation_method='median', 
                                variables = ['BedroomAbvGr', 'GarageYrBlt', 'LotFrontage']) ), 

    # Feature Engineering
    ("OrdinalEncoder", OrdinalEncoder(encoding_method="arbitrary",
                                variables = ["GarageFinish", "KitchenQual", "BsmtExposure", "BsmtFinType1"]) ),

    ("LogTransformer", vt.LogTransformer(
                         variables = ["LotFrontage", "1stFlrSF", "LotArea", "GrLivArea"]) ),

    ("PowerTransformer", vt.PowerTransformer(
                         variables = ["GarageArea", "GrLivArea", "BsmtFinSF1", "MasVnrArea", "OpenPorch", "BsmtUnfSF"]) ),

    ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.8, selection_method="variance") )
    ])

    return pipeline_base